In [1]:
# this is a file that we put together with spit and hopes in order to see if the model is working in real time
# the model is able to identify objects: "car" , "truck", "roadsign" ,"traffic light"

In [2]:
import numpy as np
from PIL import ImageGrab, Image
import cv2 as cv
import os
from time import sleep
import pygetwindow as gw

In [3]:
class WindowCapture:
    def __init__(self, window_title):
        self.window_title = window_title
        self.window = self._get_window()
        self.w, self.h = self.get_window_size()

    def _get_window(self):
        windows = gw.getWindowsWithTitle(self.window_title)
        if windows:
            return windows[0]
        else:
            raise Exception(f"No window found with title: {self.window_title}")

   

    def get_screenshot(self):
        bbox = (self.window.left, self.window.top, self.window.right, self.window.bottom)
        printscreen = np.array(ImageGrab.grab(bbox=bbox))
        img = cv.cvtColor(printscreen, cv.COLOR_BGR2RGB) # screenshot color space adjustment
        return img
    
    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while True:
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.png")
            sleep(1)
    
    def get_window_size(self):
        return (self.window.width, self.window.height)

In [4]:
class ImageProcessor:
    W = 0
    H = 0
    net = None
    ln = None
    classes = {}
    colors = []

    def __init__(self, img_size, cfg_file, weights_file):
        np.random.seed(42)
        self.net = cv.dnn.readNetFromDarknet(cfg_file, weights_file)
        self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.ln = self.net.getLayerNames()
        self.ln = [self.ln[i-1] for i in self.net.getUnconnectedOutLayers()]
        self.W = img_size[0]
        self.H = img_size[1]
        
        with open('yolov4-tiny/obj.names', 'r') as file:
            lines = file.readlines()
        for i, line in enumerate(lines):
            self.classes[i] = line.strip()
        
        # If you plan to utilize more than six objects, please include additional colors in this list.
        self.colors = [
            (0, 0, 255), 
            (0, 255, 0), 
            (255, 0, 0), 
            (255, 255, 0), 
            (255, 0, 255), 
            (0, 255, 255)
        ]
        

    def proccess_image(self, img):

        blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outputs = self.net.forward(self.ln)
        outputs = np.vstack(outputs)
        
        coordinates = self.get_coordinates(outputs, 0.5)

        self.draw_identified_objects(img, coordinates)

        return coordinates

    def get_coordinates(self, outputs, conf):

        boxes = []
        confidences = []
        classIDs = []

        for output in outputs:
            scores = output[5:]
            
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > conf:
                x, y, w, h = output[:4] * np.array([self.W, self.H, self.W, self.H])
                p0 = int(x - w//2), int(y - h//2)
                boxes.append([*p0, int(w), int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)

        if len(indices) == 0:
            return []

        coordinates = []
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            coordinates.append({'x': x, 'y': y, 'w': w, 'h': h, 'class': classIDs[i], 'class_name': self.classes[classIDs[i]]})
        return coordinates

    def draw_identified_objects(self, img, coordinates):
        for coordinate in coordinates:
            x = coordinate['x']
            y = coordinate['y']
            w = coordinate['w']
            h = coordinate['h']
            classID = coordinate['class']
            
            color = self.colors[classID]
            
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, self.classes[classID], (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv.imshow('window',  img)


In [5]:
#Make your game window smaller cause the capture program captures a general area on the screen.

In [ ]:
window_name = "Euro Truck Simulator 2" #Make sure that your game window name matches with here. 
                                         #Sometimes the window names are completely different than what you might think.
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg" #config name
weights_file_name = "yolov4-tiny-custom_last.weights" #trained models .weight files

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)

while(True):
    
    ss = wincap.get_screenshot()
    
    if cv.waitKey(1) == ord('q'): #QUIT with Q
        cv.destroyAllWindows()
        break

    coordinates = improc.proccess_image(ss)
    
    for coordinate in coordinates:
        print(coordinate)
    print()
    
  

print('Finished.')